# Import Library

In [1]:
import kagglehub
import os, datetime
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt
import numpy as np

# Data Downloading

In [2]:
path = kagglehub.dataset_download("xhlulu/140k-real-and-fake-faces")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/140k-real-and-fake-faces


# Data Loading

In [3]:
base_dir = "/kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/"
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
valid_dir = os.path.join(base_dir, 'valid')
train_datagen = ImageDataGenerator(rescale=1.0/255.)
test_datagen = ImageDataGenerator(rescale=1.0/255.)
valid_datagen = ImageDataGenerator(rescale=1.0/255.)
train_generator = train_datagen.flow_from_directory(
    train_dir, batch_size=32, class_mode='binary', target_size=(150, 150))
validation_generator = valid_datagen.flow_from_directory(
    valid_dir, batch_size=32, class_mode='binary', target_size=(150, 150))
test_generator = test_datagen.flow_from_directory(
    test_dir, batch_size=32, class_mode='binary', target_size=(150, 150), shuffle=False)

Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


# DenseNet121 Model

In [6]:
base_model = DenseNet121(include_top=False, input_shape=(150, 150, 3), weights='imagenet')
base_model.trainable = False
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model Training

In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    validation_steps=50,
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 443s 133ms/step - accuracy: 0.7165 - loss: 0.5606 - val_accuracy: 0.7756 - val_loss: 0.4630
Epoch 2/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 218s 66ms/step - accuracy: 0.7733 - loss: 0.4727 - val_accuracy: 0.7856 - val_loss: 0.4604
Epoch 3/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 261s 84ms/step - accuracy: 0.7885 - loss: 0.4517 - val_accuracy: 0.8050 - val_loss: 0.4360
Epoch 4/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 211s 68ms/step - accuracy: 0.7969 - loss: 0.4359 - val_accuracy: 0.8313 - val_loss: 0.3948
Epoch 5/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 256s 66ms/step - accuracy: 0.8076 - loss: 0.4229 - val_accuracy: 0.8006 - val_loss: 0.4274
Epoch 6/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 207s 66ms/step - accuracy: 0.8097 - loss: 0.4128 - val_accuracy: 0.8213 - val_loss: 0.4148
Epoch 7/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 262s 66ms/step - accuracy: 0.8168 - loss: 0.4050 - val_accuracy: 0.8144 - val_loss: 0.4030
Epoch 8/10
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 262s 84ms/step - accuracy:

# Plot Results

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

# Evalaution Metrics Results

In [9]:
test_generator.reset()
y_pred_probs = model.predict(test_generator, verbose=1)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()
y_true = test_generator.classes
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
print("Evaluation Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-Score : {f1:.4f}")

625/625 ━━━━━━━━━━━━━━━━━━━━ 87s 122ms/step
Evaluation Metrics:
Accuracy : 0.8295
Precision: 0.8139
Recall   : 0.8543
F1-Score : 0.8336
